In [1]:
import pandas as pd
import numpy as np
import os,sys,pickle,glob,gc
from tqdm import tqdm

In [2]:
import findspark
os.environ['PYSPARK_PYTHON'] = '/usr/bin/python3' #'/home/antslab/anaconda3/bin/python'
findspark.init()
import pyspark
# from pyspark import SparkContext
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
import pandas as pd
import numpy as np
from tqdm import tqdm
from pyspark.sql import Row, SQLContext, SparkSession, window
from pyspark import SparkConf, SparkContext
# Import data types
from pyspark.sql.types import *
import  pyspark.sql.functions as F
conf = SparkConf()
conf.set("spark.local.dir", "/mnt/ssd240g/data/Leo_Spark_Home/tmp")
conf.set("spark.executor.cores","12")
conf.set("spark.driver.memory","79g")
conf.set("spark.executor.memory", "89g")
conf.set("spark.memory.offHeap.enabled","true")
conf.set("spark.memory.offHeap.size","75g")
conf.set("spark.driver.extraJavaOptions","-Xss19990m")
conf.set("spark.driver.extraJavaOptions","-Xms79g")
conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")
conf.set("spark.sql.execution.arrow.enabled", "true")
conf.set("spark.kubernetes.pyspark.pythonVersion","3")
conf.set("spark.sql.shuffle.partitions",99990)
conf.set("spark.driver.maxResultSize", "20g")
conf.set("spark.sql.debug.maxToStringFields", 190000)
conf.set("spark.sql.hive.filesourcePartitionFileCacheSize",99*1024*1024*1024)
spark = SparkSession.builder.master("local").appName("Test_spark").config(conf=conf).getOrCreate()
# sc = SparkContext(conf=conf)
spark.conf.set("spark.sql.execution.arrow.enabled", True)
import databricks.koalas as ks
ks.options.display.max_rows = 20
ks.set_option('compute.max_rows', None)
# ks.set_option('compute.ops_on_diff_frames', True)
ks.set_option('compute.default_index_type', 'distributed')
print(ks.__version__)

0.32.0


In [3]:
# in_parquet = '../../data_hdd4t/pcap_process/pcap_spark/2020_01_09/中華電信/parquet/2020_01_09_country_session.parquet/'
in_parquet = '../../data_hdd4t/pcap_process/pcap_spark/2020_01_10/中華電信/parquet/2020_01_10_country_session_withtime.parquet/'

# df = pd.read_parquet(in_parquet)
# df
kf = ks.read_parquet(in_parquet)
# df = kf

In [4]:
kf.columns

Index(['frame.time_epoch', 'session_lifetime', 'session_packets',
       'tcp.window_size_value.avg', 'tcp.window_size_value.max', 'tcp.srcport',
       'tcp.dstport', 'tcp.flags.ack', 'tcp.flags.fin', 'tcp.flags.push',
       'tcp.flags.reset', 'tcp.flags.syn', 'ip.src', 'ip.dst', 'ip.ttl.avg',
       'ip.proto', 'frame.protocols.common', 'frame.len.sum', 'city',
       'subdivisions', 'latitude', 'longitude', 'domain', 'isp', 'network',
       'country'],
      dtype='object')

In [6]:
%%time
kf1 = kf[['tcp.dstport','domain','ip.src','frame.time_epoch']] # 需要加入time
allgby = kf1.groupby(['tcp.dstport','domain'])

CPU times: user 2.15 ms, sys: 3.14 ms, total: 5.29 ms
Wall time: 18.8 ms


In [ ]:
allgby